# Spiking Neural Network trained on CIFAR-10-DVS

In [1]:
import numpy
print(f"Numpy version: {numpy.version.version}") # sanity check. Should be 1.26.4

Numpy version: 1.26.4


In [ ]:
import tonic
import tonic.transforms as transforms
import snntorch as snn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

Note: you may need to restart the kernel to use updated packages.


/home/kalebkei/AIResearch/Event-Based/SNN_CIFAR10-DVS/.EventCifarVenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the Dataset

In [3]:
# Get sensor dimensions for NMNIST dataset
sensor_size = tonic.datasets.CIFAR10DVS.sensor_size

# Define transformations for event data
# - Denoise removes isolated events over 10000μs window
# - ToFrame converts events to frames with 1000μs time window
frame_transform = transforms.Compose([transforms.Denoise(filter_time=10000),
                                      transforms.ToFrame(sensor_size=sensor_size,
                                                         time_window=1000)
                                     ])

# Load NMNIST dataset with transformations
dataset = tonic.datasets.CIFAR10DVS(save_to='./tmp/data', transform=frame_transform)

# Print dataset information
print(f"Number of training samples: {len(dataset)}")
print(f"Classes: {dataset.classes}")
print(f"Sensor size: {dataset.sensor_size}")
# Print the shape of the first sample
print(f"Shape of first sample: {dataset[0][0].shape}")

Number of training samples: 10000
Classes: {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Sensor size: (128, 128, 2)
Shape of first sample: (1304, 2, 128, 128)


### Prep the dataset

In [ ]:
# Split the dataset into training and test sets
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)

# Print dataset information
print(f"Number of training samples: {len(trainset)}")
print(f"Number of test samples: {len(testset)}")
print(f"Sensor size: {sensor_size}")
print(f"Number of classes: {dataset.classes}")
# Print the first sample in the training set
print(f"First sample in training set: {trainset[0]}")
# Print the first sample in the test set# Print the first sample in the training set

In [ ]:
# Create the dataloaders
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

# Print the first batch of training data
for batch in train_loader:
    print(f"Batch shape: {batch[0].shape}")
    print(f"Batch labels: {batch[1]}")
    break
# Print the first batch of test data
for batch in test_loader:
    print(f"Batch shape: {batch[0].shape}")
    print(f"Batch labels: {batch[1]}")
    break

## Define the model